In [4]:
# Library Imports.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Allows plots to appear directly in the notebook.
%matplotlib inline

from patsy import dmatrices
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

import pickle

In [5]:
# Read CSV files into Data Frames:
weather = pd.read_csv('working_weatherHistoryResampled_asof_2022.04.06.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)
station = pd.read_csv('working_stationStateResampled_asof_2022.04.06.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)

In [6]:
weather.shape

(684, 7)

In [7]:
station.shape

(75240, 6)

In [8]:
weather.head()

,weatherHour,latitude,longitude,description,temp,humidity,wind_speed
0,2022-02-22 12:00:00,53.35,-6.26,few clouds,282.81,71,10.80
1,2022-02-24 10:00:00,53.35,-6.26,scattered clouds,276.31,80,9.77
2,2022-03-01 09:00:00,53.35,-6.26,clear sky,277.21,86,0.98
3,2022-03-01 10:00:00,53.35,-6.26,scattered clouds,279.13,83,1.78
4,2022-03-01 11:00:00,53.35,-6.26,broken clouds,280.61,80,3.65


In [9]:
station.head()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes
0,143661,1,2022-02-22 12:00:00,OPEN,12,18
1,143662,2,2022-02-22 12:00:00,OPEN,18,2
2,143663,3,2022-02-22 12:00:00,OPEN,20,13
3,143664,4,2022-02-22 12:00:00,OPEN,22,13
4,143665,5,2022-02-22 12:00:00,OPEN,38,2


In [10]:
stationWeather = pd.merge(station, weather, how="left", on=["weatherHour"])

In [11]:
stationWeather.head()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed
0,143661,1,2022-02-22 12:00:00,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.8
1,143662,2,2022-02-22 12:00:00,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.8
2,143663,3,2022-02-22 12:00:00,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.8
3,143664,4,2022-02-22 12:00:00,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.8
4,143665,5,2022-02-22 12:00:00,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.8


In [12]:
stationWeather.tail()

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed
75235,218896,106,2022-04-04 10:00:00,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84
75236,218897,107,2022-04-04 10:00:00,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84
75237,218898,108,2022-04-04 10:00:00,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84
75238,218899,109,2022-04-04 10:00:00,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84
75239,218900,110,2022-04-04 10:00:00,OPEN,28,2,53.35,-6.26,broken clouds,285.57,86,10.84


In [13]:
stationWeather.dtypes

ID                         int64
stationId                  int64
weatherHour               object
status                    object
available_bike_stands      int64
available_bikes            int64
latitude                 float64
longitude                float64
description               object
temp                     float64
humidity                   int64
wind_speed               float64
dtype: object

In [14]:
stationWeather[['weatherDate', 'weatherHour']] = stationWeather['weatherHour'].str.split(' ', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate
0,143661,1,12:00:00,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
1,143662,2,12:00:00,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
2,143663,3,12:00:00,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
3,143664,4,12:00:00,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
4,143665,5,12:00:00,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10:00:00,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75236,218897,107,10:00:00,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75237,218898,108,10:00:00,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04
75238,218899,109,10:00:00,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04


In [15]:
stationWeather[['weatherHour', 'weatherMinute', 'weatherSecond']] = stationWeather['weatherHour'].str.split(':', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00


In [16]:
stationWeather[['weatherYear', 'weatherMonth', 'weatherDay']] = stationWeather['weatherDate'].str.split('-', expand=True)
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,weatherYear,weatherMonth,weatherDay
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04


In [17]:
stationWeather['cal_bike_stands'] = stationWeather.available_bike_stands + stationWeather.available_bikes
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,weatherYear,weatherMonth,weatherDay,cal_bike_stands
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22,30
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22,20
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22,33
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22,35
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,02,22,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04,20
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04,40
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04,40
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,04,04,28


In [18]:
# Change description to category type for numeric encoding
stationWeather['description'] = stationWeather['description'].astype('category')
stationWeather['weatherYear'] = stationWeather['weatherYear'].astype('int64')
stationWeather['weatherMonth'] = stationWeather['weatherMonth'].astype('int64')
stationWeather['weatherDay'] = stationWeather['weatherDay'].astype('int64')
stationWeather.dtypes

ID                          int64
stationId                   int64
weatherHour                object
status                     object
available_bike_stands       int64
available_bikes             int64
latitude                  float64
longitude                 float64
description              category
temp                      float64
humidity                    int64
wind_speed                float64
weatherDate                object
weatherMinute              object
weatherSecond              object
weatherYear                 int64
weatherMonth                int64
weatherDay                  int64
cal_bike_stands             int64
dtype: object

In [19]:
# Catergorical Encoding
stationWeather['num_desc'] = stationWeather['description'].cat.codes
stationWeather

,ID,stationId,weatherHour,status,available_bike_stands,available_bikes,latitude,longitude,description,temp,humidity,wind_speed,weatherDate,weatherMinute,weatherSecond,weatherYear,weatherMonth,weatherDay,cal_bike_stands,num_desc
0,143661,1,12,OPEN,12,18,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,2,22,30,2
1,143662,2,12,OPEN,18,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,2,22,20,2
2,143663,3,12,OPEN,20,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,2,22,33,2
3,143664,4,12,OPEN,22,13,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,2,22,35,2
4,143665,5,12,OPEN,38,2,53.35,-6.26,few clouds,282.81,71,10.80,2022-02-22,00,00,2022,2,22,40,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75235,218896,106,10,OPEN,3,17,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,4,4,20,0
75236,218897,107,10,OPEN,29,11,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,4,4,40,0
75237,218898,108,10,OPEN,37,3,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,4,4,40,0
75238,218899,109,10,OPEN,4,24,53.35,-6.26,broken clouds,285.57,86,10.84,2022-04-04,00,00,2022,4,4,28,0


In [20]:
# Checking correlations for all the contious features
stationWeather.corr()

,ID,stationId,available_bike_stands,available_bikes,latitude,longitude,temp,humidity,wind_speed,weatherYear,weatherMonth,weatherDay,cal_bike_stands,num_desc
ID,1.000000,1.461928e-03,0.000109,-0.002508,NaN,NaN,2.304128e-01,-1.411975e-01,-2.389431e-01,NaN,5.216452e-01,5.341456e-01,-0.002657,8.602108e-03
stationId,0.001462,1.000000e+00,-0.012042,-0.042931,NaN,NaN,8.552269e-16,-6.187358e-16,-2.477227e-16,NaN,3.648792e-15,2.072456e-15,-0.064043,2.009989e-16
available_bike_stands,0.000109,-1.204187e-02,1.000000,-0.680484,NaN,NaN,1.946609e-02,-1.353588e-02,1.009944e-03,NaN,-1.736075e-03,1.703976e-03,0.574030,-1.162099e-03
available_bikes,-0.002508,-4.293149e-02,-0.680484,1.000000,NaN,NaN,-2.569532e-02,2.459045e-02,-4.632753e-03,NaN,4.845055e-03,-7.324583e-03,0.209393,-3.387794e-03
latitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
longitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
temp,0.230413,8.552269e-16,0.019466,-0.025695,NaN,NaN,1.000000e+00,-4.675910e-01,1.368502e-01,NaN,-1.390100e-01,3.793883e-01,-0.002737,-3.380526e-02
humidity,-0.141197,-6.187358e-16,-0.013536,0.024590,NaN,NaN,-4.675910e-01,1.000000e+00,-1.789565e-01,NaN,-8.338770e-03,-1.352310e-01,0.009416,-4.910726e-02
wind_speed,-0.238943,-2.477227e-16,0.001010,-0.004633,NaN,NaN,1.368502e-01,-1.789565e-01,1.000000e+00,NaN,-7.770332e-03,-2.457230e-01,-0.003829,1.195065e-02
weatherYear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Random Forest

In [21]:
X = stationWeather[['temp', 'humidity', 'wind_speed', 'num_desc',\
                    'cal_bike_stands',\
                    'weatherMonth', 'weatherDay','weatherHour']]
y = stationWeather.available_bikes

<h2>Training</h2>

In [22]:
randomForest_predictions = np.empty(0)
#y_test = pd.DataFrame()
y_test = np.empty(0)

for i in range(1, 111):
    # Select just current station
    stWeathTemp = stationWeather.loc[stationWeather['stationId'] == 1]
    
    X = stWeathTemp[['temp', 'humidity', 'wind_speed', 'num_desc',\
                    'cal_bike_stands',\
                    'weatherMonth', 'weatherDay','weatherHour']]
    y = stWeathTemp.available_bikes
    
    X_train, X_test, y_train, y_test_temp = train_test_split(X, y, test_size=0.3, random_state=0)

    # X_train_one_station
    random_forest = RandomForestRegressor(n_estimators = 50, random_state = 42)
    random_forest.fit(X_train, y_train)

    # Serialize model object into a file called model.pkl on disk using pickle
    with open('pickles/station' + str(i) + '_randomForest_model.pkl', 'wb') as handle:
        pickle.dump(random_forest, handle, pickle.HIGHEST_PROTOCOL)
        
    # We test each model as we build it..
    randomForest_predictions = np.append(randomForest_predictions, random_forest.predict(X_test))
    # y_test = pd.concat([y_test, y_test_temp])
    y_test = np.append(y_test, y_test_temp)

In [23]:
print(np.shape(randomForest_predictions))
print(np.shape(y_test))

(22660,)
(22660,)


<h2>Testing</h2>

In [24]:
df = pd.DataFrame({'Actual':y_test, 'Predicted':randomForest_predictions})
df

,Actual,Predicted
0,27.0,27.38
1,24.0,24.66
2,13.0,13.44
3,14.0,12.62
4,20.0,17.22
...,...,...
22655,25.0,23.72
22656,19.0,19.58
22657,16.0,21.02
22658,22.0,22.32


In [25]:
# Metrics for the Model
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, randomForest_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, randomForest_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, randomForest_predictions)))

Mean Absolute Error: 3.7216504854368933
Mean Squared Error: 22.604879611650485
Root Mean Squared Error: 4.75445891891501


In [26]:
# Calculate the absolute errors
errors = abs(randomForest_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
print('MAPE:', mape)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 3.72 degrees.
MAPE: [ 1.40740741  2.75        3.38461538 ... 31.375       1.45454545
  3.07692308]
Accuracy: 62.15 %.


In [27]:
# Function to output evaluation metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [28]:
printMetrics(y_test, randomForest_predictions)


MAE:  3.7216504854368933
RMSE:  4.75445891891501
R2:  0.6194684513599413
